In [1]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, TensorBoard
from collections import Counter
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(2020)
import pandas as pd

Using TensorFlow backend.
/home/rukesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rukesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rukesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rukesh/.local/lib/python3.6/site-packages/tensorflow/python

In [2]:
# set default parameters
BATCH_SIZE = 64
NUM_EPOCHS = 300
HIDDEN_UNITS = 256
MAX_INPUT_SEQ_LENGTH = 20
MAX_TARGET_SEQ_LENGTH = 20
MAX_VOCAB_SIZE = 100
DATA_PATH = '/home/rukesh/Downloads/file.csv'
TENSORBOARD = 'TensorBoard/'
WEIGHT_FILE_PATH = '/home/rukesh/Documents/final_year/FYP/encode_decodechat/chat_mod.h5'


In [3]:
#/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model

input_counter = Counter()
target_counter = Counter()

DATA_PATH

'/home/rukesh/Downloads/file.csv'

In [4]:
# read the data
# with open(DATA_PATH, 'r',encoding="latin-1") as f:
#     df=f.read()   
# print(df)
# # rows = df.split('\n')
# # lines = [row.split(' +++$+++ ')[-1] for row in rows]
# input_texts = []
# target_texts = []

# prev_words = []
# # print(lines)


dataset = pd.read_csv(DATA_PATH)

In [5]:
dataset.head()

,human,reply
0,Namastae sir.,Namastae sir/mam. How can we help you?
1,My internet is not working well for a past few...,Please mention your username and the number si...
2,My username is rukesh_home.,How far are you from your router sir/mam?
3,I am next to my router.,How many lights are glowing in your router?
4,I have got the black nokia router. The lights ...,Can you please turn off the router and check a...


In [6]:
data1 = dataset.loc[:,['human']]

In [7]:
data2 = dataset.loc[:,['reply']]

In [8]:
list_of_humansentences = [sent_list[0] for sent_list in data1.values]
list_of_replysentences = [sent_list[0] for sent_list in data2.values]

In [9]:
single_data = list_of_humansentences + list_of_replysentences
input_texts = []
target_texts = []
prev_words = []

In [10]:
for line in single_data:

    next_words = [w.lower() for w in nltk.word_tokenize(line)]
    print(next_words)
    if len(next_words) > MAX_TARGET_SEQ_LENGTH:
        next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

    if len(prev_words) > 0:
        input_texts.append(prev_words)
        for w in prev_words:
            input_counter[w] += 1
        target_words = next_words[:]
        target_words.insert(0, 'START')
        target_words.append('END')
        for w in target_words:
            target_counter[w] += 1
        target_texts.append(target_words)

    prev_words = next_words


['namastae', 'sir', '.']
['my', 'internet', 'is', 'not', 'working', 'well', 'for', 'a', 'past', 'few', 'days', '.']
['my', 'username', 'is', 'rukesh_home', '.']
['i', 'am', 'next', 'to', 'my', 'router', '.']
['i', 'have', 'got', 'the', 'black', 'nokia', 'router', '.', 'the', 'lights', 'glowing', 'are', 'auth', ',', 'power', ',', 'link', ',', 'wlan', '.']
['sure', 'sir/mam', '.', 'thanks', 'a', 'lot', '.']
['hello', ',', 'worldlink', '.']
['could', 'you', 'please', 'mention', 'the', 'customer', 'accounts', 'number', '?']
['thank', 'you', 'so', 'much', 'dear', '.', 'have', 'a', 'beautiful', 'day', '.']
['hello', '.']
['please', 'mention', 'the', 'accounts', 'number', '?']
['thank', 'you', '.']
['hello', ',', 'provide', 'me', 'the', 'account', 'number', '.']
['hey', ',', 'worldlink', '.']
['please', 'mention', 'the', 'accounts', 'number', '?']
['thank', 'you', '.']
['oi', 'worldlink', '.']
['tero', 'tauko', '.']
['provide', 'me', 'the', 'account', 'number', '.']
['hi', ',', 'worldlink', '

['sir', ',', 'fup', 'is', 'right', 'that', 'every', 'isp', 'reserve', '.', 'we', 'are', 'sure', 'sooner', 'or', 'later', 'it', 'is', 'going', 'to', 'be', 'implemented', 'by', 'any', 'isps', 'according', 'to', 'their', 'needs', 'and', 'requirements', '.', 'so', 'on', 'with', 'the', 'increasing', 'number', 'of', 'clients', 'in', 'worldlink', 'we', 'intend', 'to', 'provide', 'the', 'best', 'possible', 'internet', 'experience', 'to', 'all', 'our', 'customers', '.', 'for', 'this', ',', 'we', 'rely', 'on', 'customers', 'being', 'fair', 'in', 'the', 'way', 'they', 'use', 'the', 'internet', '.', 'this', '``', 'fair', 'usage', 'policy', "''", 'is', 'here', 'to', 'give', 'us', 'an', 'option', 'that', 'if', 'usage', 'of', 'any', 'unlimited', 'customer', 'is', 'excessive', 'over', 'a', 'period', 'of', 'time', 'to', 'the', 'point', ',', 'it', 'impacts', 'on', 'other', 'users', 'experience', 'then', 'the', 'bandwidth', 'will', 'be', 'capped', 'to', 'fallback', 'speed', 'and', 'is', 'effective', 'onl

In [11]:

# for line in lines:

#     next_words = [w.lower() for w in nltk.word_tokenize(line)]
#     print(next_words)
#     if len(next_words) > MAX_TARGET_SEQ_LENGTH:
#         next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

#     if len(prev_words) > 0:
#         input_texts.append(prev_words)
#         for w in prev_words:
#             input_counter[w] += 1
#         target_words = next_words[:]
#         target_words.insert(0, 'START')
#         target_words.append('END')
#         for w in target_words:
#             target_counter[w] += 1
#         target_texts.append(target_words)

#     prev_words = next_words


In [12]:
input_texts[1]

['my',
 'internet',
 'is',
 'not',
 'working',
 'well',
 'for',
 'a',
 'past',
 'few',
 'days',
 '.']

In [13]:
target_texts[0]

['START',
 'my',
 'internet',
 'is',
 'not',
 'working',
 'well',
 'for',
 'a',
 'past',
 'few',
 'days',
 '.',
 'END']

In [14]:
# encode the data
input_word2idx = dict()
target_word2idx = dict()
for idx, word in enumerate(input_counter.most_common(MAX_VOCAB_SIZE)):
    input_word2idx[word[0]] = idx + 2
for idx, word in enumerate(target_counter.most_common(MAX_VOCAB_SIZE)):
    target_word2idx[word[0]] = idx + 1

input_word2idx['PAD'] = 0
input_word2idx['UNK'] = 1
target_word2idx['UNK'] = 0

input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])
target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_encoder_tokens = len(input_idx2word)
num_decoder_tokens = len(target_idx2word)

np.save('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-input-word2idx.npy', input_word2idx)
np.save('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-input-idx2word.npy', input_idx2word)
np.save('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-target-word2idx.npy', target_word2idx)
np.save('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-target-idx2word.npy', target_idx2word)

encoder_input_data = []

encoder_max_seq_length = 0
decoder_max_seq_length = 0

for input_words, target_words in zip(input_texts, target_texts):
    encoder_input_wids = []
    for w in input_words:
        w2idx = 1
        if w in input_word2idx:
            w2idx = input_word2idx[w]
        encoder_input_wids.append(w2idx)

    encoder_input_data.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_words), decoder_max_seq_length)

context = dict()
context['num_encoder_tokens'] = num_encoder_tokens
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length

np.save('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-context.npy', context)


In [15]:
# custom function to generate batches

def generate_batch(input_data, output_text_data):
    num_batches = len(input_data) // BATCH_SIZE
    while True:
        for batchIdx in range(0, num_batches):
            start = batchIdx * BATCH_SIZE
            end = (batchIdx + 1) * BATCH_SIZE
            encoder_input_data_batch = pad_sequences(input_data[start:end], encoder_max_seq_length)
            decoder_target_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            decoder_input_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            for lineIdx, target_words in enumerate(output_text_data[start:end]):
                for idx, w in enumerate(target_words):
                    w2idx = 0
                    if w in target_word2idx:
                        w2idx = target_word2idx[w]
                    decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                    if idx > 0:
                        decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
            yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch



In [16]:
# Compiling and training

encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=HIDDEN_UNITS,
                              input_length=encoder_max_seq_length, name='encoder_embedding')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

json = model.to_json()
open('/home/rukesh/PycharmProjects/generativechatbot/ChatBot/model/word-architecture.json', 'w').write(json)

X_train, X_test, y_train, y_test = train_test_split(encoder_input_data, target_texts, test_size=0.2, random_state=42)

train_gen = generate_batch(X_train, y_train)
test_gen = generate_batch(X_test, y_test)

train_num_batches = len(X_train) // BATCH_SIZE
test_num_batches = len(X_test) // BATCH_SIZE

checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
tbCallBack = TensorBoard(log_dir=TENSORBOARD, histogram_freq=0, write_graph=True, write_images=True)

model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_num_batches,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=test_gen,
                    validation_steps=test_num_batches,
                    callbacks=[checkpoint, tbCallBack ])

model.save_weights(WEIGHT_FILE_PATH)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/300
15/15 [==============================] - 2s 130ms/step - loss: 1.8191 - accuracy: 0.4939 - val_loss: 1.6432 - val_accuracy: 0.1392

Epoch 2/300
15/15 [==============================] - 1s 98ms/step - loss: 1.5293 - accuracy: 0.5762 - val_loss: 1.5676 - val_accuracy: 0.6508
Epoch 3/300
15/15 [==============================] - 1s 95ms/step - loss: 1.4803 - accuracy: 0.6636 - val_loss: 1.5392 - val_accuracy: 0.6508
Epoch 4/300
15/15 [==============================] - 1s 98ms/step - loss: 1.4484 - accuracy: 0.6636 - val_loss: 1.5082 - val_accuracy: 0.6508
Epoch 5/300
15/15 [==============================] - 2s 100ms/step - loss: 1.4209 - accuracy: 0.6638 - val_loss: 1.4799 - val_accuracy: 0.6508
Epoch 6/300
15/15 [==============================] - 1s 94ms/step - loss: 1.3920 - accuracy: 0.6690 - val_loss: 1.4548 - val_accuracy: 0.6652
Epoch 7/300
15/15 [=============================

15/15 [==============================] - 1s 88ms/step - loss: 0.4228 - accuracy: 0.5733 - val_loss: 1.0272 - val_accuracy: 0.5178
Epoch 51/300
15/15 [==============================] - 1s 87ms/step - loss: 0.4076 - accuracy: 0.5896 - val_loss: 1.0528 - val_accuracy: 0.4995
Epoch 52/300
15/15 [==============================] - 1s 91ms/step - loss: 0.4023 - accuracy: 0.6106 - val_loss: 1.0525 - val_accuracy: 0.4896
Epoch 53/300
15/15 [==============================] - 1s 89ms/step - loss: 0.3859 - accuracy: 0.6251 - val_loss: 1.0659 - val_accuracy: 0.4815
Epoch 54/300
15/15 [==============================] - 1s 88ms/step - loss: 0.3783 - accuracy: 0.5778 - val_loss: 1.0646 - val_accuracy: 0.5440
Epoch 55/300
15/15 [==============================] - 1s 89ms/step - loss: 0.3638 - accuracy: 0.6123 - val_loss: 1.0636 - val_accuracy: 0.4922
Epoch 56/300
15/15 [==============================] - 1s 89ms/step - loss: 0.3502 - accuracy: 0.6081 - val_loss: 1.0883 - val_accuracy: 0.4865
Epoch 57/300

15/15 [==============================] - 2s 103ms/step - loss: 0.0622 - accuracy: 0.6493 - val_loss: 1.4258 - val_accuracy: 0.4474
Epoch 108/300
15/15 [==============================] - 1s 91ms/step - loss: 0.0613 - accuracy: 0.6473 - val_loss: 1.4312 - val_accuracy: 0.4444
Epoch 109/300
15/15 [==============================] - 1s 86ms/step - loss: 0.0605 - accuracy: 0.6471 - val_loss: 1.4371 - val_accuracy: 0.4489
Epoch 110/300
15/15 [==============================] - 1s 86ms/step - loss: 0.0597 - accuracy: 0.6464 - val_loss: 1.4497 - val_accuracy: 0.4515
Epoch 111/300
15/15 [==============================] - 2s 101ms/step - loss: 0.0590 - accuracy: 0.6460 - val_loss: 1.4576 - val_accuracy: 0.4590
Epoch 112/300
15/15 [==============================] - 2s 102ms/step - loss: 0.0584 - accuracy: 0.6449 - val_loss: 1.4671 - val_accuracy: 0.4553
Epoch 113/300
15/15 [==============================] - 2s 100ms/step - loss: 0.0578 - accuracy: 0.6432 - val_loss: 1.4657 - val_accuracy: 0.4517
Ep

Epoch 164/300
15/15 [==============================] - 2s 101ms/step - loss: 0.0471 - accuracy: 0.6148 - val_loss: 1.5374 - val_accuracy: 0.4330
Epoch 165/300
15/15 [==============================] - 1s 91ms/step - loss: 0.0463 - accuracy: 0.6181 - val_loss: 1.5431 - val_accuracy: 0.4283
Epoch 166/300
15/15 [==============================] - 2s 101ms/step - loss: 0.0457 - accuracy: 0.6134 - val_loss: 1.5503 - val_accuracy: 0.4297
Epoch 167/300
15/15 [==============================] - 2s 107ms/step - loss: 0.0453 - accuracy: 0.6140 - val_loss: 1.5547 - val_accuracy: 0.4268
Epoch 168/300
15/15 [==============================] - 1s 87ms/step - loss: 0.0449 - accuracy: 0.6130 - val_loss: 1.5624 - val_accuracy: 0.4271
Epoch 169/300
15/15 [==============================] - 2s 107ms/step - loss: 0.0446 - accuracy: 0.6129 - val_loss: 1.5669 - val_accuracy: 0.4261
Epoch 170/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0443 - accuracy: 0.6125 - val_loss: 1.5730 - val_accur

Epoch 221/300
15/15 [==============================] - 1s 88ms/step - loss: 0.0394 - accuracy: 0.6021 - val_loss: 1.7355 - val_accuracy: 0.4171
Epoch 222/300
15/15 [==============================] - 1s 88ms/step - loss: 0.0393 - accuracy: 0.6039 - val_loss: 1.7403 - val_accuracy: 0.4145
Epoch 223/300
15/15 [==============================] - 1s 88ms/step - loss: 0.0393 - accuracy: 0.6022 - val_loss: 1.7398 - val_accuracy: 0.4162
Epoch 224/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0393 - accuracy: 0.6043 - val_loss: 1.7446 - val_accuracy: 0.4134
Epoch 225/300
15/15 [==============================] - 1s 91ms/step - loss: 0.0392 - accuracy: 0.6020 - val_loss: 1.7441 - val_accuracy: 0.4148
Epoch 226/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0392 - accuracy: 0.6041 - val_loss: 1.7488 - val_accuracy: 0.4131
Epoch 227/300
15/15 [==============================] - 1s 94ms/step - loss: 0.0391 - accuracy: 0.6015 - val_loss: 1.7484 - val_accuracy:

Epoch 278/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0379 - accuracy: 0.6020 - val_loss: 1.8374 - val_accuracy: 0.4060
Epoch 279/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0378 - accuracy: 0.5998 - val_loss: 1.8398 - val_accuracy: 0.4079
Epoch 280/300
15/15 [==============================] - 1s 90ms/step - loss: 0.0378 - accuracy: 0.6019 - val_loss: 1.8402 - val_accuracy: 0.4060
Epoch 281/300
15/15 [==============================] - 1s 91ms/step - loss: 0.0378 - accuracy: 0.5998 - val_loss: 1.8426 - val_accuracy: 0.4077
Epoch 282/300
15/15 [==============================] - 1s 92ms/step - loss: 0.0378 - accuracy: 0.6023 - val_loss: 1.8430 - val_accuracy: 0.4065
Epoch 283/300
15/15 [==============================] - 1s 92ms/step - loss: 0.0378 - accuracy: 0.5998 - val_loss: 1.8455 - val_accuracy: 0.4072
Epoch 284/300
15/15 [==============================] - 1s 91ms/step - loss: 0.0378 - accuracy: 0.6022 - val_loss: 1.8458 - val_accuracy:

In [17]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, 20, 256)      26112       encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 101)    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 525312      encoder_embedding[0][0]          
____________________________________________________________________________________________

In [18]:
from keras.models import load_model
model.load_weights(WEIGHT_FILE_PATH)

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, 20, 256)      26112       encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 101)    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 525312      encoder_embedding[0][0]          
____________________________________________________________________________________________